# CLIP TEXT BASED 77x768 BASED

In [1]:
import os
import sys
import h5py
from dotenv import load_dotenv 
from pathlib import Path
load_dotenv()


project_root = Path(os.path.abspath('')).parent.parent
sys.path.append(str(project_root))



intermediate_output_dir = project_root / 'data'

NIH_CXR14_DATASET_DIR = os.getenv("NIH_CXR14_DATASET_DIR")
print(NIH_CXR14_DATASET_DIR)



/home/yasin/Lfstorage/datasets/nih-cxr14


In [2]:
import h5py
import pickle
import pandas as pd
import numpy as np

In [3]:
from src.utils import Unet2DConditionalTrainer, TrainConfig, get_validation_samples, create_validation_dataloader

/home/yasin/Lfstorage/Projects/cxr-diffusion/.venv/lib/python3.10/site-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-03-09 03:30:49.916935: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741480249.939576  934738 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741480249.946434  934738 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-09 03:30:49.970953: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.


# Load Latents
For the training optimization and efficient in this training we used prepared latents. They generated from `nih-cxr14-dataset` with using `VaeProccessor` which in this repository

In [ ]:
latent

In [ ]:
from diffusers import DDPMScheduler, UNet2DConditionModel
from torch.utils.data import Dataset

import numpy as np
import os
import torch
class EmbeddingLatentDataset(Dataset):
    def __init__(self, latent_dir, text_embeddings_path):
        """
        Dataset to load latent vectors and text embeddings for medical images.
        
        Args:
            latent_dir (str): Directory containing latent vector H5 files
            text_embeddings_path (str): Path to pickled text embeddings map
        """
        # Load text embeddings from pickle file
        with open(text_embeddings_path, 'rb') as f:
            self.text_embeddings_map = pickle.load(f)

        self.latent_dir = latent_dir
        self.latent_maps = self._load_latent_maps()
        
        # Create a flat list of all available image IDs for indexing
        self.image_ids = []
        self.file_indices = []  # Store (file_name, index) tuples for fast lookup
        
        for file_name, id_map in self.latent_maps.items():
            for image_id, idx in id_map.items():
                self.image_ids.append(image_id)
                self.file_indices.append((file_name, idx))
        
        print(f"Found {len(self.image_ids)} valid images with both latents and text embeddings")
    
    def _load_latent_maps(self):
        """
        Load mapping from image IDs to their location in latent files.
        
        Returns:
            dict: Nested dictionary mapping file names to image IDs to indices
        """
        latent_files = os.listdir(self.latent_dir)
        latent_maps = {}

        text_embedding_ids = list(self.text_embeddings_map.keys())
        print(f"Total text embeddings available: {len(text_embedding_ids)}")
        
        matches_found = 0
        
        for latent_file in latent_files:
            if not latent_file.endswith('.h5'):
                continue
                
            with h5py.File(os.path.join(self.latent_dir, latent_file), 'r') as f:
                image_ids = f['Image index'][:]
                # Convert byte strings to regular strings if needed
                if isinstance(image_ids[0], bytes):
                    image_ids = [idx.decode('utf-8') for idx in image_ids]
                    
                latent_maps[latent_file] = {}
                for i, image_id in enumerate(image_ids):
                    if image_id in text_embedding_ids:
                        latent_maps[latent_file][image_id] = i
                        matches_found += 1
        
        print(f"Found {matches_found} matches between latent files and text embeddings")
        return latent_maps
    
    def __len__(self):
        """Return the total number of available image pairs"""
        return len(self.image_ids)
    
    def __getitem__(self, index):
        """
        Get a latent vector and text embedding pair by index.
        
        Args:
            index (int): Index in the dataset
            
        Returns:
            tuple: (latent, text_embed) pair as torch tensors
        """
        # Get the image ID and file location info for this index
        image_id = self.image_ids[index]
        file_name, file_index = self.file_indices[index]
        
        # Load the latent vector from the H5 file
        with h5py.File(os.path.join(self.latent_dir, file_name), 'r') as f:
            latent = f['latents'][file_index]
        
        # Get the corresponding text embedding
        text_embed = self.text_embeddings_map[image_id]
        
        # Convert to torch tensors
        latent_tensor = torch.tensor(latent, dtype=torch.float32)
        
        # Convert text embedding to tensor if it's not already
        if not isinstance(text_embed, torch.Tensor):
            text_embed_tensor = torch.tensor(text_embed, dtype=torch.float32)
        else:
            text_embed_tensor = text_embed
            
        # Make sure text embedding has the right shape for the UNet model
        # If it's a 2D tensor [seq_len, hidden_dim], we're good
        # If it's a 3D tensor with batch dim [1, seq_len, hidden_dim], we squeeze it
        if len(text_embed_tensor.shape) == 3 and text_embed_tensor.shape[0] == 1:
            text_embed_tensor = text_embed_tensor.squeeze(0)

        ## complete the seq length to 77 for each text embedding
        seq_len = 77
        if text_embed_tensor.shape[0] < seq_len:
            pad_len = seq_len - text_embed_tensor.shape[0]
            padding = torch.zeros((pad_len, text_embed_tensor.shape[1]))
            text_embed_tensor = torch.cat([text_embed_tensor, padding], dim=0)


        return latent_tensor, text_embed_tensor


/home/yasin/Lfstorage/Projects/cxr-diffusion/.venv/lib/python3.10/site-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-03-02 14:01:16.816808: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740913276.841590 3480791 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740913276.849015 3480791 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 14:01:16.874316: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.


In [ ]:
clip_text_embeddings_path = intermediate_output_dir / "clip_text_embeds.pkl"
latent_dir = os.path.join(NIH_CXR14_DATASET_DIR, "vae_latents2")


In [4]:
dataset = EmbeddingLatentDataset(latent_dir, clip_text_embeddings_path)
len(dataset)

Total text embeddings available: 48311
Found 48311 matches between latent files and text embeddings
Found 48311 valid images with both latents and text embeddings


48311

In [5]:
config = TrainConfigV2(
    batch_size=4,
    mixed_precision=True,
    learning_rate=1e-4,
    num_epochs=20,
    gradient_accumulation_steps=64,  # Effective batch size of 128
    scheduler_type="cosine",
    early_stopping_patience=5,
    use_timestep_weights=True
)

In [6]:
unet = UNet2DConditionModel(act_fn="silu",
                                    attention_head_dim=8,
                                    center_input_sample=False,
                                    downsample_padding=1,
                                    flip_sin_to_cos=True,
                                    freq_shift=0,
                                    mid_block_scale_factor=1,
                                    norm_eps=1e-05,
                                    norm_num_groups=32,
                                    sample_size=64, # generated samples are 512x512
                                    in_channels=4, 
                                    out_channels=4, 
                                    layers_per_block=2, 
                                    block_out_channels=(320, 640, 1280, 1280), 
                                    down_block_types=(
                                    "CrossAttnDownBlock2D",
                                    "CrossAttnDownBlock2D",
                                    "CrossAttnDownBlock2D",
                                    "DownBlock2D"), 
                                    up_block_types=("UpBlock2D",
                                    "CrossAttnUpBlock2D",
                                    "CrossAttnUpBlock2D",
                                    "CrossAttnUpBlock2D"),
                                    cross_attention_dim=768
                                )

In [7]:
optimizer = torch.optim.AdamW(unet.parameters(), lr=config.learning_rate)
noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

In [8]:
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.batch_size, num_workers=config.num_workers, shuffle=True, pin_memory=True)


In [9]:
validation_samples = get_validation_samples_v2(dataset, num_samples=16)

In [10]:
print(validation_samples[0].shape)

torch.Size([77, 768])


In [11]:
import tensorboard
trainer = Unet2DConditionalTrainerV2(
    unet=unet,
    train_config=config,
    noise_scheduler=noise_scheduler,
    optimizer=optimizer,
)



Log directory is output/logs
Output directory is output


In [ ]:
trainer.train(dataloader = train_dataloader,  validation_samples = validation_samples)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/12078 [00:00<?, ?it/s]

Epoch 1/20 - Avg loss: 0.158840, Avg PSNR: 22.58
Saving model to output/best/best_model.safetensors
New best model with loss: 0.158840
Saving model to output/checkpoints/epoch_1_model.safetensors
Generating validation samples for epoch 1...
Initialized VaeProcessor for sample generation


Generating sample 1/16:   0%|          | 0/50 [00:00<?, ?it/s]

Generating sample 2/16:   0%|          | 0/50 [00:00<?, ?it/s]

...

Generating sample 15/16:   0%|          | 0/50 [00:00<?, ?it/s]

Generating sample 16/16:   0%|          | 0/50 [00:00<?, ?it/s]

Saved sample grid to output/samples/final_samples.png
Loading best model
Loaded model from output/best/best_model.safetensors
Resuming from epoch 18, global step 229482


UNet2DConditionModel(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0-1): 2 x Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): Linear(in_features=320, out_features=320, bias=False)
                (to_v): Linear(in_features=320, out_fe

: 